# Working with Jobs
The intent of this tutorial is to help familiarize yourself with _jobs_. Unity provides users the ability to execute _jobs_ to produce data. The steps below will showcase how a job is typically submitted to the Unity Platform.

#### Set up imports and predefined variables

In [ ]:
from utils import api

app_name = "job-hello_world:develop"

## 1. Deploy a job
With an ADES environment ready to accept our job, we can submit a job along with any input data that is needed. In this example case, none are needed so `inputs` is blank. The response will provide a Job ID that will store in a variable called `job_id` for use later.

In [ ]:
data = {
  "mode": "async",
  "response": "document",
  "inputs": [
  ],
  "outputs": [
    {
      "id": "output",
      "transmissionMode": "reference"
    }
  ]
}

try:

    # Store Job ID to use in future steps
    job_id = api.submit_job(app_name, data)

    # If the job submission is successful, print a success message along with the returned JOB-ID
    print("\nJob Submission Successful!\nJOB ID: {}\n".format(job_id))

except requests.exceptions.HTTPError as e:
    # An error has occurred, print the error message that was generated
    print(e.reason)

## 2. Get the job status
The code below will demonstrate how one can check the status of a job. 

In this example, we will use the predfined application name, and the job ID that was returned previously.

In [ ]:
try:
    
    job_status = api.get_job_status(app_name, job_id)
    
    # Print the job status
    print("\nStatus for job \"{}\": {}\n".format(job_id, job_status))
    
except request.exceptions.HTTPError as e:
    # An error has occurred, print the error message that was generated
    print(e.reason)
        

## 3. Get job results
Now that we've monitored the status of a job, and verified that is has completed, we can retreive information about where the generated data is located. 

In this example, we will use the predfined application name, and the job ID that was returned previously.

In [ ]:
try:

    job_data = api.get_job_result(app_name, job_id)

    for data in job_data:
        print("ID: " + data['id'])
        print("Type: " + data['mimeType'])
        print("Location: " + data['href'])

except request.exceptions.HTTPError as e:
    print(e.reason)